# **Launch Sites Locations Analysis with Folium**


## Introduction 

**Capstone Project for 'IBM Professional Certificate in Data Science with Python'**   
In this project, I constructed a machine learning model that can predict whether Falcon 9 will land successfuly in the first stage. [Falcon 9](https://www.spacex.com/vehicles/falcon-9/) is classified as a medium-lift partially reusable rocket, used to launch hefty communications and satellites into Earth orbit or ferry austronaouts to and from the International Space Station. As of April 2022, SpaceX offers Falcon 9 rocket launches for [USD 62 million](https://www.nbcnews.com/science/space/space-launch-costs-growing-business-industry-rcna23488), which means around USD 1,200 per pound of payload. For comparison, per pound cost of SpaceX competitors is 3 to 5 times more expensive, whereas traditional NASA space shuttles, retired in 2011, cost an average of [USD 1.6 billion](https://aerospace.csis.org/data/space-launch-to-low-earth-orbit-how-much-does-it-cost/) per flight. SpaceX is able to provide rocket launches for unprecedented low prices, because it can reuse the first stage, which significantly reduces the demand for new cores. Therefore, determining whether the first stage will land, helps to estimate the cost of a launch. This information can also be used if another company wants to bid against SpaceX for a rocket launch. For this project, I use API requests to pull data from <code>https://api.spacexdata.com/v4</code> and webscraping to collect data from Wikipedia. 

--- 

# Part 6: Launch Sites Locations Analysis with Folium 

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`. 


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

Let's first import required Python packages for this lab:

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:
[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb) 


## 0. Imports 

In [1]:
import pandas as pd

import requests
import io
import folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon 
from math import radians, sin, cos, sqrt, atan2 

In [2]:
# URL of the dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Make a GET request to fetch the raw CSV data
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    # Convert the content of the response into a BytesIO object
    spacex_csv_file = io.BytesIO(response.content)
    
    # Read the CSV data into a pandas DataFrame
    spacex_df = pd.read_csv(spacex_csv_file)
    
else:
    print("Failed to retrieve the CSV file. Status code:", response.status_code) 

In [3]:
spacex_df.tail(10) 

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


In [6]:
# Unique Launch Sites
spacex_df['Launch Site'].unique()


array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [10]:
# check the data types in spacex_df
spacex_df.dtypes

Flight Number          int64
Date                  object
Time (UTC)            object
Booster Version       object
Launch Site           object
Payload               object
Payload Mass (kg)    float64
Orbit                 object
Customer              object
Landing Outcome       object
class                  int64
Lat                  float64
Long                 float64
dtype: object

## 1. Mark all launch sites on a map 

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 

Above coordinates are just plain numbers that do not give any intuitive insights about where are those launch sites, unles you are unusually good at geography.  

Let's visualize those locations by pinning them on a map. We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas. 


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10) 

We use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=5000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(25,25),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:black;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker) 

Now, let's add a circle for each launch site in data frame `launch_sites`


In [14]:
# Get unique launch sites and their corresponding latitude and longitude
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# Display the list of launch sites
print(launch_sites)  

    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1   VAFB SLC-4E  34.632834 -120.610745
2    KSC LC-39A  28.573255  -80.646895
3  CCAFS SLC-40  28.563197  -80.576820


In [15]:
# Initial the map
initial_coords = [28.562302, -80.577356]  # Coordinates of CCAFS LC-40
site_map = folium.Map(location=initial_coords, zoom_start=5)  

For each launch site, we add a Circle object based on its coordinate values: Lat, Long. In addition, we add labels: Launch Site names

In [16]:

# Initialize a map centered around the first launch site
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Launch sites data
launch_sites = [
    {"name": "VAFB SLC-4E", "coordinates":  [34.632834, -120.610745]},
    {"name": "KSC LC-39A", "coordinates":   [28.573255, -80.646895]},
    {"name": "CCAFS LC-40", "coordinates":  [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "coordinates": [28.563197, -80.576820]}
]

# Loop through each launch site to add a Circle and a Marker
for site in launch_sites:
    # Extract the name and coordinates
    name, coordinates = site['name'], site['coordinates']
    
    # Create a Circle
    circle = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(name))

    # Create a Marker with a popup label
    marker = folium.map.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{name}</b></div>',
        )
    )

    # Add Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map 
site_map  

Now, we can explore the map by zoom-in/out the marked areas. There are 3 luanch sites in Florida and 1 in California. Two of the launch sites in Flordia are almost overlapping. We also observe that 

*   All launch sites are in proximity to the Equator line 
*   All launch sites are in very close proximity to the coast

Also please try to explain your findings.


## 2. Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have higher success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

First, we create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate. 

Let's first create a `MarkerCluster` object 

In [17]:
# Create a 'MarkerCluster' object  
marker_cluster = MarkerCluster()  

# Create a new launch_sites dataframe 
launch_sites = spacex_df[['Flight Number', 'Date', 'Launch Site', 'class', 'Lat', 'Long']].copy() 

# Add a new column 'marker_color' to the dataframe  
launch_sites['marker_color'] = launch_sites['class'].apply(lambda x: 'green' if x == 1 else 'red') 

For each launch result in `spacex_df` data frame, we add a `folium.Marker` to `marker_cluster`

In [18]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through the launch_sites DataFrame to create markers
for index, row in launch_sites.iterrows():
    # Extract information for each launch
    flight_number = row['Flight Number']
    date = row['Date']
    launch_site = row['Launch Site']
    marker_color = row['marker_color']
    coordinates = [row['Lat'], row['Long']]

    # Create a popup string
    popup_string = f"Flight Number: {flight_number}<br>Date: {date}<br>Launch Site: {launch_site}"

    # Create a marker for each launch
    marker = folium.Marker(
        location=coordinates,
        popup=popup_string,
        icon=folium.Icon(color=marker_color)
    )

    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
# Assuming you have a Folium map object already created named 'site_map'
site_map.add_child(marker_cluster)

# Display the map
site_map 


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## 3. Calculate the distances between a launch site to its proximities 

Next, we explore and analyze the geographic environment of the launch sites  

`MousePosition`  
  
First, add a MousePosition function on the map to get coordinate for a mouse over a point on the map.   
As such, we can easily find the coordinates of any object of interests by hovering over that point with the mouse pointer.   
We can see the coordinates on the top-right corner of the map. 


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map 

As we zoom in to our map and explore the geographic terrain around the launch sites, we notice some common attributes among them: 
* They are close to the coastline 
* They are not close to urban centers 
* They have highways and railways in close proximity    

We can move our mouse and identify the coordinates (shown on the top-left) of these goegraphic objects. Then we can use the coordinates to measure the distances. Below I've marked down the coordinates of a nearby point on the coast line and the nearest point on the railway line. Then I add those coordinates to my code to measure the distances to the launch sites. 

A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>

In [20]:
# Define the coastline point coordinates
coastline_lat = 28.56345
coastline_lon = -80.56799

# Launch sites data
launch_sites_data = [
    {"name": "CCAFS LC-40", "coordinates":  [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "coordinates": [28.563197, -80.576820]}
]

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calculate distance from each site to the coastline point
closest_site = None
min_distance = float('inf')
for site in launch_sites_data:
    distance = calculate_distance(site['coordinates'][0], site['coordinates'][1], coastline_lat, coastline_lon)
    print(f"Distance from {site['name']} to coastline: {distance} km")
    if distance < min_distance:
        min_distance = distance
        closest_site = site['name'] 

Distance from CCAFS LC-40 to coastline: 0.923853931054433 km
Distance from CCAFS SLC-40 to coastline: 0.8630789362768558 km


Below map inclused a `PolyLine` between the launch sites and the coastline. 


In [22]:
# Initialize the map centered around CCAFS LC-40 and CCAFS SLC-40 with a closer zoom level
map_center = [28.562302, -80.577356]
site_map = folium.Map(location=map_center, zoom_start=16)  # Increased zoom level

# Coastline point coordinates
coastline_coords = [coastline_lat, coastline_lon]

# Add red circles for CCAFS LC-40 and CCAFS SLC-40 only
for site in launch_sites_data:
    folium.Circle(
        location=site['coordinates'],
        radius=500,  # radius in meters
        color='red',
        fill=True
    ).add_to(site_map) 

# Function to add polyline and distance marker (same as before)
def add_polyline_and_distance_marker(map_obj, site_coords, coastline_coords, site_name):
    # Polyline
    folium.PolyLine(locations=[site_coords, coastline_coords], color='yellow').add_to(map_obj)

    # Distance calculation
    distance = calculate_distance(site_coords[0], site_coords[1], coastline_coords[0], coastline_coords[1])

    # Midpoint for placing the distance marker
    midpoint = [(site_coords[0] + coastline_coords[0]) / 2, (site_coords[1] + coastline_coords[1]) / 2]

    # Distance Marker
    folium.map.Marker(
        midpoint,
        icon=DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 16; color:#000000;"><b>{distance:.2f} KM to {site_name}</b></div>',
        )
    ).add_to(map_obj)


# Add polylines and distance markers for CCAFS LC-40 and CCAFS SLC-40
add_polyline_and_distance_marker(site_map, launch_sites_data[0]['coordinates'], coastline_coords, launch_sites_data[0]['name'])
add_polyline_and_distance_marker(site_map, launch_sites_data[1]['coordinates'], coastline_coords, launch_sites_data[1]['name'])


# Display the map
site_map 


Similarly, I drew a line between the launch sites and the railway line. 

In [23]:
# Define the coastline point coordinates
railway_lat = 28.57161
railway_lon = -80.58534

# Launch sites data
launch_sites_data = [
    {"name": "CCAFS LC-40", "coordinates":  [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "coordinates": [28.563197, -80.576820]}
]

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calculate distance from each site to the coastline point
closest_site = None
min_distance = float('inf')
for site in launch_sites_data:
    distance = calculate_distance(site['coordinates'][0], site['coordinates'][1], railway_lat, railway_lon)
    print(f"Distance from {site['name']} to railway line: {distance} km")
    if distance < min_distance:
        min_distance = distance
        closest_site = site['name'] 

Distance from CCAFS LC-40 to railway line: 1.2962321961005472 km
Distance from CCAFS SLC-40 to railway line: 1.2523603848119942 km


In [25]:
# Initialize the map centered around CCAFS LC-40 and CCAFS SLC-40 with a closer zoom level
map_center = [28.562302, -80.577356]
site_map = folium.Map(location=map_center, zoom_start=15)  # Increased zoom level

# Coastline point coordinates
railway_coords = [railway_lat, railway_lon]

# Add red circles for CCAFS LC-40 and CCAFS SLC-40 only
for site in launch_sites_data:
    folium.Circle(
        location=site['coordinates'],
        radius=500,  # radius in meters
        color='red',
        fill=True
    ).add_to(site_map) 

# Function to add polyline and distance marker (same as before)
def add_polyline_and_distance_marker(map_obj, site_coords, railway_coords, site_name):
    # Polyline
    folium.PolyLine(locations=[site_coords, railway_coords], color='yellow').add_to(map_obj)

    # Distance calculation
    distance = calculate_distance(site_coords[0], site_coords[1], railway_coords[0], railway_coords[1])

    # Midpoint for placing the distance marker
    midpoint = [(site_coords[0] + railway_coords[0]) / 2, (site_coords[1] + railway_coords[1]) / 2]

    # Distance Marker
    folium.map.Marker(
        midpoint,
        icon=DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 16; color:#000000;"><b>{distance:.2f} KM to {site_name}</b></div>',
        )
    ).add_to(map_obj)


# Add polylines and distance markers for CCAFS LC-40 and CCAFS SLC-40
add_polyline_and_distance_marker(site_map, launch_sites_data[0]['coordinates'], railway_coords, launch_sites_data[0]['name'])
add_polyline_and_distance_marker(site_map, launch_sites_data[1]['coordinates'], railway_coords, launch_sites_data[1]['name'])


# Display the map
site_map 


These visualizations helped us gain a better understanding of the environment surrounding our launch sites.   
**Next:**  We will build a Plotly Dashbaord with detailed information on launch records.